In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_raw = pd.read_excel('./data/2014-2019 PM10 LisAvLib.xlsx')
df_raw.columns = ['Date', 'value']
df_raw['Day'] = pd.DatetimeIndex(df_raw['Date']).date

FileNotFoundError: [Errno 2] No such file or directory: './data/2014-2019 PM10 LisAvLib.xlsx'

In [ ]:
df_raw

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(df_raw.Date,df_raw.value)

In [ ]:
df = df_raw.groupby('Day')[['Day','value']].mean().reset_index()
df.Day = pd.to_datetime(df.Day)

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(df.Day,df.value)
plt.ylim(0,200)

In [ ]:
import plotly.express as px
fig = px.line(x=df.index, y= df.value)
fig.show()

In [ ]:
df['interpol'] = df['value'].interpolate()

In [ ]:
oct_15 = df[(df['Day']>= "2015-09-01") & (df['Day']<= "2015-10-31")]
#oct_14 = df[(df['Day']>= "2014-09-01") & (df['Day']<= "2014-10-31")]
#oct_16 = df[(df['Day']>= "2016-09-01") & (df['Day']<= "2016-10-31")]

plt.figure(figsize=(16,5))
plt.plot(list(oct_15.value))
plt.plot(list(oct_15.interpol),'.')
#plt.plot(list(oct_16.value))
#plt.plot(list(oct_14.value))

In [ ]:
df.set_index('Day', inplace=True)

In [ ]:
for i in range(2015,2016):
    df_year= df[(df.index >= f"{i}-01-01") & (df.index<= f"{i}-12-31")]

    from statsmodels.tsa.seasonal import seasonal_decompose
    decomp_add = seasonal_decompose(df_year.interpol, model="additive")
    decomp_mult = seasonal_decompose(df_year.interpol, model="multiplicative")

    fig = decomp_add.plot();
    fig.suptitle(f'Year {i}')
    fig.set_size_inches(16, 8)

In [ ]:
#from statsmodels.tsa.stattools import adfuller
#print('p-value: ', adfuller(y)[1])
#print("Reject")

In [ ]:
decomp_add = seasonal_decompose(df.interpol, model="additive")

In [ ]:
df['noise'] = decomp_add.resid
df['season'] = decomp_add.seasonal
df['trend'] = decomp_add.trend

In [ ]:
df.interpol

In [ ]:
decomp_add.resid

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Original Series
fig, axes = plt.subplots(3, 2, figsize=(13,10))
axes[0, 0].plot(df.noise); axes[0, 0].set_title('Noise Series')
plot_acf(df.noise,lags = 15,missing = 'drop', ax=axes[0, 1])

# Diff Series
axes[1, 0].plot(df.noise.diff()); axes[1, 0].set_title('Differentiated Series')
plot_acf(df.noise.diff(),lags = 15,missing = 'drop', ax=axes[1, 1])

# Diff Series
axes[2, 0].plot(df.noise.diff().diff()); axes[2, 0].set_title('Differentiated Series')
plot_acf(df.noise.diff().diff(),lags = 15,missing = 'drop', ax=axes[2, 1])
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(13,3))
axes[0].plot(df.interpol.diff(7)); axes[0].set_title('Without season Series')
plot_acf(df.noise,lags = 15,missing = 'drop', ax=axes[1]);
#print('p-value: ', adfuller(df.interpol.diff(7).dropna())[1])

In [ ]:
fig, ax = plt.subplots(3, figsize=(12,9))
ax[0] = plot_acf(df.noise, missing = 'drop', ax=ax[0], lags=15)
ax[1] = plot_pacf(df.noise.dropna(), ax=ax[1], lags=15)
ax[2].plot(df.noise)

In [ ]:
from pmdarima import auto_arima
model = pm.auto_arima(df.interpol.dropna(),
                      m=7, # frequency of series. Important to choose manually
                      seasonal=True,
                      stepwise=True,  # only check most interesting combination. False will take way longer.
                      start_p=0, max_p=4,
                      start_d=0, max_d=4,
                      start_q=0, max_q=4,
                      start_P=0, max_P=4,
                      D = 1,
                      start_Q=0, max_Q=2,
                      njobs=-1, trace=True, error_action='ignore',  suppress_warnings=True,information_criterion = 'bic')
print(model.summary())

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model_auto = SARIMAX(df.interpol.dropna(),order=(1,0,1),freq='D',seasonal_order=(4,1,0,7),
                     enforce_stationarity=False, enforce_invertibility=False,).fit()
model_auto.summary()

In [ ]:
start = '2014-01-01'
end = '2019-12-31'
pred = model_auto.predict(start,end)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Original Data",
    x=df.index, y=df.value
))

fig.add_trace(go.Scatter(
    name="Fitted Model",
    x=df.index, y=pred
))